In [1]:
import pandas as pd
import plotly.express as px

In [28]:
class read_gtfs(object):
    """a class to read gtfs object
       """
    def __init__(self, update_date):
        self.agency=pd.read_csv('data/'+update_date+'/agency.txt', sep=",")
        self.calen_date=pd.read_csv('data/'+update_date+'/calendar_dates.txt', sep=",")
        self.calendar=pd.read_csv('data/'+update_date+'/calendar.txt', sep=",")
        self.rout=pd.read_csv('data/'+update_date+'/routes.txt', sep=",")
        self.shape=pd.read_csv('data/'+update_date+'/shapes.txt', sep=",")
        self.stop_time=pd.read_csv('data/'+update_date+'/stop_times.txt', sep=",")
        self.stop=pd.read_csv('data/'+update_date+'/stops.txt', sep=",")
        self.trip=pd.read_csv('data/'+update_date+'/trips.txt', sep=",")


In [53]:
latest_gtfs=read_gtfs('2020-5-29')
gtfs_19_9=read_gtfs('2019-9-27')



In [59]:
#show number of trip
print('2019-9-17: trip number',gtfs_19_9.trip.shape[0])
print('2020-5-29: trip number',latest_gtfs.trip.shape[0])




2019-9-17: trip number 9232
2020-5-29: trip number 7349


In [61]:
#show number of route
print('2019-9-17: route number',gtfs_19_9.rout.shape[0])
print('2020-5-29: route number',latest_gtfs.rout.shape[0])


2019-9-17: route number 76
2020-5-29: route number 45


In [87]:
df=gtfs_19_9.stop_time
df1=df[df['trip_id']==180166]
df2=df1.merge(gtfs_19_9.stop,on='stop_id',how='left')
df2.head()
lat_center=df2['stop_lat'].mean()
lon_center=df2['stop_lon'].mean()

In [92]:
#show trip 180166

fig = px.line_mapbox(df2, lat="stop_lat", lon="stop_lon",  zoom=3, height=300)

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=10, mapbox_center_lat = lat_center,mapbox_center_lon = lon_center,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()